In [1]:
import os
import numpy as np
import pandas as pd
import corus
from tqdm import tqdm

np.random.seed(42)

## 1. Load `lenta-ru-news` / split

#### 1.2 Load raw data

In [2]:
DATA_SIZE = 100_000 # cut off size for faster performance

path = 'lenta-ru-news.csv.gz'

if not os.path.exists(path):
    !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
else:
    print("Already loaded.")

Already loaded.


In [3]:
records = corus.load_lenta(path)
def parse(record) -> dict[str, str]:
    return dict(title=record.title, text=record.text, topic=record.topic)
df = pd.DataFrame(list(tqdm(map(lambda r: parse(r), records), desc='load lenta')))
df = df.sample(DATA_SIZE, random_state=42)
df.head(10)

load lenta: 739351it [00:21, 34350.95it/s]


,title,text,topic
153198,EgyptAir объявила о подорожании билетов,Египетский перевозчик EgyptAir сообщил о возмо...,Путешествия
169154,Глава Красногорского района Подмосковья ушел в...,Глава Красногорского района Московской области...,Россия
83745,Милонов предложил запретить россиянам сидеть в...,Депутат Виталий Милонов внес в Госдуму законоп...,Россия
10029,Женщинам в детородном возрасте разрешили посещ...,Верховный суд Индии разрешил женщинам в фертил...,Мир
6445,Россиянам пообещали дешевый хлеб,Россиянам не стоит бояться роста цен на хлеб —...,Экономика
45187,МОК наказал Мутко,Российский вице-премьер Виталий Мутко пожизнен...,Спорт
399176,Жара резко увеличила энергопотребление в Москве,Потребление электроэнергии в Москве в последни...,Экономика
66430,Федор Емельяненко и Милонов снимутся во «Лжи М...,"На Урале снимут картину «Ложь Матильды», посвя...",Культура
78735,Трамп высмеял противившихся увольнению директо...,Президент США Дональд Трамп опубликовал подбор...,Мир
102670,В Москве подтвердили передачу Сербии шести ист...,Весной 2017 года Сербия получит шесть истребит...,Силовые структуры


#### 1.2 Preprocessing

##### 1.2.1 Clear up data

In [4]:
import re
from nltk.corpus import stopwords
import pymorphy3


morph = pymorphy3.MorphAnalyzer()
russian_stopwords = set(stopwords.words('russian'))

In [5]:
import functools
from typing import Callable


@functools.lru_cache(maxsize=100_000)
def normalize(token):
    return morph.parse(token)[0].normal_form

@functools.lru_cache(maxsize=100_000)
def normalize_with_pos(token):
    parse = morph.parse(token)[0]
    return f'{parse.normal_form}_{parse.tag.POS}'


def preprocess_text(text: str, normalize: Callable[[str], str] = normalize) -> str:
    text = re.sub(r'[^а-яё\s]', '', text.lower())
    return ' '.join(
        filter(
            lambda token: token not in russian_stopwords,
            map(
                normalize,
                text.split()
            )
        )
    )

In [6]:
df['processed_text'] = list(tqdm(map(
    preprocess_text, df['text'].to_list()), desc='preprocess text', total=len(df)))

preprocess text: 100%|██████████| 100000/100000 [01:32<00:00, 1081.01it/s]


In [7]:
df['processed_text_with_pos'] = list(tqdm(map(
    lambda text: preprocess_text(text, normalize_with_pos), df['text'].to_list()), desc='preprocess text', total=len(df)))

preprocess text: 100%|██████████| 100000/100000 [01:34<00:00, 1055.49it/s]


##### 1.2.2 Drop not representative classes

In [8]:
MIN_TOPIC_FREQ = int(.01*len(df))
assert MIN_TOPIC_FREQ > 3, 'Too small MIN_TOPIC_FREQ for split train/val/test'
print(f'Minimal topic frequency: {MIN_TOPIC_FREQ}')

Minimal topic frequency: 1000


In [9]:
print('Topics with counts before preprocessing')
df['topic'].value_counts()

Topics with counts before preprocessing


topic
Россия               21871
Мир                  18494
Экономика            10737
Спорт                 8632
Культура              7337
Наука и техника       7129
Бывший СССР           7100
Интернет и СМИ        6181
Из жизни              3718
Дом                   2891
Силовые структуры     2661
Ценности              1079
Бизнес                 967
Путешествия            855
69-я параллель         178
Крым                    82
Культпросвет            45
                        23
Легпром                 10
Библиотека               8
Сочи                     1
Оружие                   1
Name: count, dtype: int64

In [10]:
from enum import Enum


class Strategy(Enum):
    """
    Тут отразил разные стратегии для решения проблемы нерепрезентативности классов
    """
    NONE = 0  # ничего не делаем
    DROP = 1  # опускаем нерепрезентативные классы
    REPLACE_WITH_OTHER = 2  # скидываем из в кучу

In [11]:
STRATEGY = Strategy.REPLACE_WITH_OTHER

match STRATEGY:
    case Strategy.NONE:
        pass
    case Strategy.DROP:
        _counts = df['topic'].value_counts()
        _keep = (_counts[_counts > MIN_TOPIC_FREQ]).index.to_list()
        df = df[df['topic'].apply(lambda x: x in _keep)]
    case Strategy.REPLACE_WITH_OTHER:
        _counts = df['topic'].value_counts()
        _keep = (_counts[_counts > MIN_TOPIC_FREQ]).index.to_list()
        df['topic'] = df['topic'].apply(
            lambda x: 'other' if x not in _keep else x)

In [12]:
print(f'Topics with counts after preprocessing with strategy `{STRATEGY.name}`')
df['topic'].value_counts()

Topics with counts after preprocessing with strategy `REPLACE_WITH_OTHER`


topic
Россия               21871
Мир                  18494
Экономика            10737
Спорт                 8632
Культура              7337
Наука и техника       7129
Бывший СССР           7100
Интернет и СМИ        6181
Из жизни              3718
Дом                   2891
Силовые структуры     2661
other                 2170
Ценности              1079
Name: count, dtype: int64

##### 1.2.3 Train/Test/Val split

In [13]:
from sklearn.model_selection import train_test_split

X = df['processed_text']

y = df['topic']

def train_test_val_split(X, y, sizes: tuple[float, float, float] = (0.6, 0.2, 0.2), random_state=42):
    train_size, test_size, val_size = sizes
    
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=test_size + val_size, stratify=y, random_state=random_state
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=test_size / (1 - train_size), stratify=y_temp, random_state=42
    )
    return X_train, X_val, X_test, y_train, y_val, y_test


X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_split(X, y)

assert len(X_train) + len(X_val) + len(X_test) == len(X)
assert len(y_train) + len(y_val) + len(y_test) == len(y)
assert np.all(sorted(y.unique()) == sorted(y_train.unique()))
assert np.all(sorted(y.unique()) == sorted(y_test.unique()))
assert np.all(sorted(y.unique()) == sorted(y_val.unique()))

print(f'Train size: {len(X_train)}, {len(X_train) / len(X):.2f}%')
print(f'Val size: {len(X_val)}, {len(X_val) / len(X):.2f}%')
print(f'Test size: {len(X_test)}, {len(X_test) / len(X):.2f}%')

Train size: 60000, 0.60%
Val size: 20000, 0.20%
Test size: 20000, 0.20%


In [14]:
X_pos = df['processed_text_with_pos']

X_pos_train, X_pos_val, X_pos_test, y_train, y_val, y_test = train_test_val_split(X_pos, y)

## 2. Train Word2Vec and Simple validation

### 2.1 Train

In [15]:
from gensim.models import Word2Vec

w2v = Word2Vec(
    sentences=X_train.apply(str.split),
    vector_size=100,
    window=5,
    min_count=10,
    workers=4
)

- `vector_size=100`: размерность вектор6ного пространства, берем 100 так как данных не настолько много как в википедии, например
- `window=5`: дефолтное значение, небольшие эксперименты показывают что не сильно меняется качество при window=5+-2, а 10 много, сильно чаще может попасть какое-то слово не из контекста (например, слово из другого предложения)
- `min_count=10`: империческое число  

### 2.2 Simple Validation

In [16]:
from typing import NamedTuple
from gensim.models.keyedvectors import KeyedVectors

class Benchmark:
    class TestScore(NamedTuple):
        name: str
        total: int
        correct: int

    class __TestAccuracy(NamedTuple):
        class Case(NamedTuple):
            words: list[str]
            correct: str
        
        name: str
        cases: list[Case]

    class __TestSimilarity(NamedTuple):
        class Case(NamedTuple):
            positive: list[str]
            negative: list[str]
            correct: str
        
        name: str
        cases: list[Case]

    __TEST_ACCURACY_: list[__TestAccuracy] = [
        __TestAccuracy(
            name="Политика",
            cases=[
                __TestAccuracy.Case(["путин", "зеленский", "лукашенко", "меркель", "телескоп"], "телескоп"),
                __TestAccuracy.Case(["россия", "германия", "франция", "китай", "космос"], "космос"),
                __TestAccuracy.Case(["выборы", "голосование", "кандидат", "бюллетень", "помидор"], "помидор")
            ]
        ),
        __TestAccuracy(
            name="Экономика",
            cases=[
                __TestAccuracy.Case(["нефть", "газ", "уголь", "рубль", "подушка"], "подушка"),
                __TestAccuracy.Case(["инфляция", "дефляция", "рецессия", "стагнация", "попкорн"], "попкорн"),
                __TestAccuracy.Case(["сбербанк", "втб", "тинькофф", "альфабанк", "шаурма"], "шаурма")
            ]
        ),
        __TestAccuracy(
            name="Спорт",
            cases=[
                __TestAccuracy.Case(["футбол", "хоккей", "теннис", "баскетбол", "кроссворд"], "кроссворд"),
                __TestAccuracy.Case(["чемпионат", "лига", "турнир", "матч", "сковородка"], "сковородка")
            ]
        ),
        __TestAccuracy(
            name="Технологии",
            cases=[
                __TestAccuracy.Case(["смартфон", "ноутбук", "планшет", "компьютер", "дирижабль"], "дирижабль"),
                __TestAccuracy.Case(["программист", "разработчик", "тестировщик", "аналитик", "кастрюля"], "кастрюля")
            ]
        )
    ]

    __TEST_SIMILARITY_: list[__TestSimilarity] = [
        __TestSimilarity(
            name="Города и страны",
            cases=[
                __TestSimilarity.Case(["россия", "москва"], ["франция"], "париж"),
                __TestSimilarity.Case(["германия", "берлин"], ["италия"], "рим")
            ]
        ),
        __TestSimilarity(
            name="Политика",
            cases=[
                __TestSimilarity.Case(["президент", "россия"], ["сша"], "байден"),
                __TestSimilarity.Case(["санкция", "евросоюз"], ["китай"], "тайвань")
            ]
        ),
        __TestSimilarity(
            name="Экономика",
            cases=[
                __TestSimilarity.Case(["нефть", "доллар"], ["евро"], "рубль"),
                __TestSimilarity.Case(["криптовалюта", "биткоин"], ["акция"], "блокчейн")
            ]
        ),
        __TestSimilarity(
            name="Спорт",
            cases=[
                __TestSimilarity.Case(["месси", "футбол"], ["хоккей"], "овечкин"),
                __TestSimilarity.Case(["чемпион", "победа"], ["поражение"], "медаль")
            ]
        ),
        __TestSimilarity(
            name="Технологии",
            cases=[
                __TestSimilarity.Case(["интеллект", "алгоритм"], ["человек"], "нейросеть"),
            ]
        )
    ]

    @classmethod
    def __bench_accuracy(cls, model: KeyedVectors, test: __TestAccuracy) -> TestScore:
        cnt = 0
        for case in test.cases:
            answer = model.doesnt_match(case.words)
            if answer == case.correct: cnt += 1
        return cls.TestScore(name=test.name, total=len(test.cases), correct=cnt)

    @classmethod
    def __bench_top_similarity(cls, model: KeyedVectors, topn: int, test: __TestSimilarity) -> TestScore:
        cnt = 0
        for case in test.cases:
            answer = model.most_similar(
                positive=case.positive,
                negative=case.negative,
                topn=topn
            )
            if any(map(lambda r: r[0] == case.correct, answer)): cnt += 1
        return cls.TestScore(name=test.name, total=len(test.cases), correct=cnt)

    
    @classmethod
    def accuracy_report(cls, model: KeyedVectors):
        report = ''
        tests = [cls.__bench_accuracy(model, test) for test in cls.__TEST_ACCURACY_]
        total = sum(map(lambda a: a.total, tests))
        correct = sum(map(lambda a: a.correct, tests))
        report += f'Total Accuracy: {correct / total:.4f} (total {total})\n'
        report += '---------------------------------------\n'
        for test in tests:
            report += f'{test.name}: {test.correct / test.total:.4f} (total {test.total})\n' 
        print(report)

    @classmethod
    def similarity_report(cls, model: KeyedVectors, topn: int = 10):
        report = ''
        tests = [cls.__bench_top_similarity(model, topn, test) for test in cls.__TEST_SIMILARITY_]
        total = sum(map(lambda a: a.total, tests))
        correct = sum(map(lambda a: a.correct, tests))
        report += f'Total Accuracy@{topn}: {correct / total:.4f} (total {total})\n'
        report += '---------------------------------------\n'
        for test in tests:
            report += f'{test.name}: {test.correct / test.total:.4f} (total {test.total})\n' 
        print(report)

In [17]:
Benchmark.accuracy_report(w2v.wv)

Total Accuracy: 0.7000 (total 10)
---------------------------------------
Политика: 1.0000 (total 3)
Экономика: 0.6667 (total 3)
Спорт: 0.0000 (total 2)
Технологии: 1.0000 (total 2)



In [18]:
tops = [5, 100, 1_000]
for top in tops:
    Benchmark.similarity_report(w2v.wv, top)

Total Accuracy@5: 0.1111 (total 9)
---------------------------------------
Города и страны: 0.5000 (total 2)
Политика: 0.0000 (total 2)
Экономика: 0.0000 (total 2)
Спорт: 0.0000 (total 2)
Технологии: 0.0000 (total 1)

Total Accuracy@100: 0.3333 (total 9)
---------------------------------------
Города и страны: 0.5000 (total 2)
Политика: 0.0000 (total 2)
Экономика: 0.5000 (total 2)
Спорт: 0.5000 (total 2)
Технологии: 0.0000 (total 1)

Total Accuracy@1000: 0.3333 (total 9)
---------------------------------------
Города и страны: 0.5000 (total 2)
Политика: 0.0000 (total 2)
Экономика: 0.5000 (total 2)
Спорт: 0.5000 (total 2)
Технологии: 0.0000 (total 1)



небольшой бенчмарк, показывает, насколько хорошо веторы описывают текст новостей. Видим, что нет смысла использовать большой топ по похожести, достаточно небольшого колличества, чтобы получить почти максимальное качество

### 3. Load navec, rusvectores

In [19]:
from navec import Navec


path = 'navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)
assert np.allclose(navec['человек'][:15], np.array([-0.13068067, -0.12051002, -0.05782367,  0.07967507,  0.08338855,
        0.59920526,  0.4020081 , -1.0838276 ,  0.12556174,  0.17060532,
        0.16637331, -0.00257014,  0.51296437,  0.17175263, -0.40394753],
      dtype=np.float32))

In [20]:
import urllib.request
import gensim


rusvectores_path = 'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'
if not os.path.exists(rusvectores_path):
    urllib.request.urlretrieve(
        "https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz",
        "ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz"
    )

model_path = 'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'
model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path)

### 4. LogisticRegression

In [21]:
from typing import Mapping, Optional
from sklearn.base import BaseEstimator, TransformerMixin

class Pooling:
    def __init__(self):
        pass

    def fit(documents: list[str]):
        pass

    def __call__(self, tokens: list[str], vectors: np.ndarray) -> np.ndarray:
        raise NotImplementedError()

class MeanPooling(Pooling):
    def __call__(self, tokens: list[str], vectors: np.ndarray) -> np.ndarray:
        return np.mean(vectors, axis=0)


class VectorizeAndPool(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            model: Mapping[str, np.ndarray],
            dim: int,
            unk: Optional[str] = None,
            pooling: Pooling = MeanPooling()):
        self.model = model
        self.dim = dim
        self.unk = unk
        self.pooling = pooling
    
    def fit(self, X, y=None):
        if isinstance(self.pooling, TransformerMixin):
            self.pooling.fit(X, y)

        return self

    def __get_vector(self, token: str) -> np.ndarray:
        """get vector and handle unknown tokens 

        :param token: 
        :type token: str
        :return: vector if token is in model, else self.unk (or zeros if unk is None)
        :rtype: np.ndarray
        """
        vec = np.zeros((self.dim,), dtype=np.float32)
        try:
            vec = self.model[token]
        except KeyError:
            if self.unk is not None:
                vec = self.model[self.unk]
        return vec

    def __get_text_vector(self, text: str) -> np.ndarray:
        tokens = text.split()
        vectors = np.array(list(map(self.__get_vector, tokens)))
        if len(vectors.shape) < 2:
            return np.zeros((self.dim,))
        return self.pooling(tokens, vectors)

    def transform(self, texts: list[str]) -> np.ndarray:
        return np.array(list(map(self.__get_text_vector, texts)))

In [22]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings('ignore', category=ConvergenceWarning)

#### 4.1. Lenta Embeddings

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lenta_vectorized = Pipeline([
    ('vec', VectorizeAndPool(
        model=w2v.wv,
        dim=w2v.vector_size,
        unk=None,
        pooling=MeanPooling()
    )),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
lenta_vectorized.fit(X_train, y_train)
y_val_pred = lenta_vectorized.predict(X_val)
print(classification_report(y_val, y_val_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.25      0.55      0.34       434
      Бывший СССР       0.64      0.76      0.70      1420
              Дом       0.66      0.85      0.74       578
         Из жизни       0.43      0.70      0.53       743
   Интернет и СМИ       0.63      0.67      0.65      1236
         Культура       0.84      0.83      0.84      1468
              Мир       0.81      0.70      0.75      3699
  Наука и техника       0.79      0.75      0.77      1426
           Россия       0.82      0.52      0.63      4374
Силовые структуры       0.26      0.65      0.37       532
            Спорт       0.96      0.95      0.95      1726
         Ценности       0.60      0.85      0.71       216
        Экономика       0.82      0.76      0.79      2148

         accuracy                           0.70     20000
        macro avg       0.65      0.73      0.67     20000
     weighted avg       0.76      0.70      0.72     2

#### 4.2. Navec Embeddings

In [24]:
navec_vectorized = Pipeline([
    ('vec', VectorizeAndPool(
        model=navec,
        dim=navec.pq.dim,
        unk='<unk>',
        pooling=MeanPooling()
    )),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
navec_vectorized.fit(X_train, y_train)
y_val_pred = navec_vectorized.predict(X_val)
print(classification_report(y_val, y_val_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.28      0.55      0.37       434
      Бывший СССР       0.72      0.83      0.77      1420
              Дом       0.66      0.87      0.75       578
         Из жизни       0.46      0.68      0.55       743
   Интернет и СМИ       0.67      0.69      0.68      1236
         Культура       0.85      0.86      0.85      1468
              Мир       0.81      0.72      0.76      3699
  Наука и техника       0.77      0.77      0.77      1426
           Россия       0.84      0.56      0.68      4374
Силовые структуры       0.29      0.66      0.40       532
            Спорт       0.96      0.96      0.96      1726
         Ценности       0.59      0.85      0.70       216
        Экономика       0.82      0.76      0.79      2148

         accuracy                           0.73     20000
        macro avg       0.67      0.75      0.69     20000
     weighted avg       0.77      0.73      0.74     2

#### 4.3. RusVector Embeddings

In [25]:
rusvec_vectorized = Pipeline([
    ('vec', VectorizeAndPool(
        model=model_ru,
        dim=300,
        unk=None,
        pooling=MeanPooling()
    )),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
rusvec_vectorized.fit(X_pos_train, y_train)
y_val_pred = rusvec_vectorized.predict(X_pos_val)
print(classification_report(y_val, y_val_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.25      0.41      0.31       434
      Бывший СССР       0.48      0.66      0.55      1420
              Дом       0.56      0.80      0.66       578
         Из жизни       0.36      0.63      0.45       743
   Интернет и СМИ       0.58      0.62      0.60      1236
         Культура       0.80      0.83      0.81      1468
              Мир       0.79      0.66      0.72      3699
  Наука и техника       0.77      0.70      0.73      1426
           Россия       0.75      0.40      0.52      4374
Силовые структуры       0.20      0.53      0.29       532
            Спорт       0.95      0.92      0.94      1726
         Ценности       0.52      0.80      0.63       216
        Экономика       0.75      0.77      0.76      2148

         accuracy                           0.65     20000
        macro avg       0.60      0.67      0.61     20000
     weighted avg       0.70      0.65      0.66     2

Получаем лучший результат с `novec`.

Гипотеза: `novec` лучше обученных Word2Vec, так как векторы учились на бОльших данных, лучше `rusvectors`, так как для `rusvectors` нужен POS, который может определяться с ошибками, из-за чего смысл текста в глазах модели будет искажен 

### 5. TfIdfWeighted

In [26]:
from collections import Counter, defaultdict

class TfIfdWeightsPooling(Pooling):
    def __init__(self):
        self.log_idf: dict[str, float] = {}

    @staticmethod
    def tf(tokens: list[str]) -> np.ndarray:
        cnts = Counter(tokens)
        total = sum(cnts.values())
        return np.array([cnts[token] / total for token in tokens], dtype=np.float32)

    def idf(self, tokens: list[str]) -> np.ndarray:
        return np.array([self.log_idf.get(token, 0.) for token in tokens], dtype=np.float32)

    def weights(self, tokens: list[str]) -> np.ndarray:
        tf_idf = self.tf(tokens) * self.idf(tokens)
        return tf_idf / np.linalg.norm(tf_idf) if max(tf_idf) > 0 else tf_idf # нормализация потому что sklearn ее тоже делает

    def fit(self, documents: list[str]):
        documents_counts = defaultdict(int)
        documents_cnt = len(documents)
        for doc in documents:
            for token in set(doc.split()):
                documents_counts[token] += 1
        self.log_idf = {token: np.log(documents_cnt / cnt) + 1 for token, cnt in documents_counts.items()}
    

    def __call__(self, tokens: list[str], vectors: np.ndarray) -> np.ndarray:
        if vectors.ndim != 2:
            return vectors
        return np.sum(vectors.T * self.weights(tokens), axis=1)


In [27]:
def test():
    from sklearn.feature_extraction.text import TfidfVectorizer

    train = [
        "Data Science is the sexiest job of the 21st century",
        "machine learning is the key for data science"
    ]

    val = [
        "machine learning is the for job"
    ]

    vectorize= TfidfVectorizer(smooth_idf=False, lowercase=False)
    pool = TfIfdWeightsPooling()
    vectorize.fit(train)
    pool.fit(train)

    left = vectorize.transform(val).data
    right = pool.weights(val[0].split())

    assert np.allclose(sorted(left), sorted(right))

test()

In [28]:
navec_weighted_vectorized = Pipeline([
    ('vec', VectorizeAndPool(
        model=navec,
        dim=navec.pq.dim,
        unk='<unk>',
        pooling=TfIfdWeightsPooling()
    )),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
navec_weighted_vectorized.fit(X_train, y_train)
y_val_pred = navec_weighted_vectorized.predict(X_val)
print(classification_report(y_val, y_val_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.00      0.00      0.00       434
      Бывший СССР       0.00      0.00      0.00      1420
              Дом       0.00      0.00      0.00       578
         Из жизни       0.00      0.00      0.00       743
   Интернет и СМИ       0.00      0.00      0.00      1236
         Культура       0.00      0.00      0.00      1468
              Мир       0.00      0.00      0.00      3699
  Наука и техника       0.00      0.00      0.00      1426
           Россия       0.00      0.00      0.00      4374
Силовые структуры       0.00      0.00      0.00       532
            Спорт       0.09      1.00      0.16      1726
         Ценности       0.00      0.00      0.00       216
        Экономика       0.00      0.00      0.00      2148

         accuracy                           0.09     20000
        macro avg       0.01      0.08      0.01     20000
     weighted avg       0.01      0.09      0.01     2

Tf-Idf взвешивание делает сильно хуже.

Гипотеза: взвешивание анрушает структуру векторного пространства word2vec и по этому мы теряем сильный сигнал

## 6. Total Comparison

In [29]:
lenta_vectorized = Pipeline([
    ('vec', VectorizeAndPool(
        model=w2v.wv,
        dim=w2v.vector_size,
        unk=None,
        pooling=MeanPooling()
    )),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
lenta_vectorized.fit(X_train, y_train)
y_test_pred = lenta_vectorized.predict(X_test)
print(classification_report(y_test, y_test_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.24      0.52      0.32       434
      Бывший СССР       0.66      0.76      0.70      1420
              Дом       0.65      0.84      0.73       578
         Из жизни       0.43      0.69      0.53       744
   Интернет и СМИ       0.61      0.67      0.64      1236
         Культура       0.83      0.82      0.82      1467
              Мир       0.82      0.70      0.75      3699
  Наука и техника       0.76      0.73      0.74      1426
           Россия       0.83      0.53      0.65      4374
Силовые структуры       0.26      0.65      0.37       532
            Спорт       0.96      0.95      0.96      1727
         Ценности       0.62      0.87      0.72       216
        Экономика       0.81      0.73      0.77      2147

         accuracy                           0.70     20000
        macro avg       0.65      0.73      0.67     20000
     weighted avg       0.76      0.70      0.71     2

In [30]:
navec_vectorized = Pipeline([
    ('vec', VectorizeAndPool(
        model=navec,
        dim=navec.pq.dim,
        unk='<unk>',
        pooling=MeanPooling()
    )),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
navec_vectorized.fit(X_train, y_train)
y_test_pred = navec_vectorized.predict(X_test)
print(classification_report(y_test, y_test_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.28      0.55      0.37       434
      Бывший СССР       0.72      0.83      0.77      1420
              Дом       0.66      0.87      0.75       578
         Из жизни       0.48      0.70      0.57       744
   Интернет и СМИ       0.65      0.71      0.68      1236
         Культура       0.85      0.84      0.85      1467
              Мир       0.83      0.73      0.78      3699
  Наука и техника       0.77      0.75      0.76      1426
           Россия       0.84      0.58      0.68      4374
Силовые структуры       0.29      0.70      0.41       532
            Спорт       0.96      0.96      0.96      1727
         Ценности       0.57      0.87      0.69       216
        Экономика       0.83      0.75      0.79      2147

         accuracy                           0.73     20000
        macro avg       0.67      0.76      0.70     20000
     weighted avg       0.78      0.73      0.74     2

In [31]:
rusvec_vectorized = Pipeline([
    ('vec', VectorizeAndPool(
        model=model_ru,
        dim=300,
        unk=None,
        pooling=MeanPooling()
    )),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
rusvec_vectorized.fit(X_pos_train, y_train)
y_test_pred = rusvec_vectorized.predict(X_pos_test)
print(classification_report(y_test, y_test_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.23      0.38      0.28       434
      Бывший СССР       0.48      0.65      0.55      1420
              Дом       0.54      0.79      0.64       578
         Из жизни       0.36      0.66      0.47       744
   Интернет и СМИ       0.55      0.62      0.58      1236
         Культура       0.82      0.80      0.81      1467
              Мир       0.80      0.66      0.72      3699
  Наука и техника       0.74      0.68      0.71      1426
           Россия       0.77      0.40      0.53      4374
Силовые структуры       0.19      0.54      0.29       532
            Спорт       0.95      0.92      0.94      1727
         Ценности       0.52      0.87      0.65       216
        Экономика       0.74      0.75      0.75      2147

         accuracy                           0.64     20000
        macro avg       0.59      0.67      0.61     20000
     weighted avg       0.70      0.64      0.65     2

Как и предпологалось, лучше всего себя показывает `novec`